# x402 Image Generation Demo

Dieses Notebook demonstriert den **x402 Payment Protocol Flow** für die ImageGen API.

## Was ist x402?

x402 ist ein HTTP-basiertes Payment Protocol:
- Server antwortet mit `402 Payment Required` + Payment-Instruktionen
- Client führt Payment durch (hier: NFT Mint)
- Client wiederholt Request mit Payment Proof
- Server verifiziert Payment on-chain und liefert Service

## Voraussetzungen

**Server:**
```bash
cd scw_js
npm run dev:x402
```
→ Server läuft auf `http://localhost:8081`

**Environment:**
- `NFT_WALLET_PRIVATE_KEY` in `.env` gesetzt
- Optimism RPC Zugriff
- BFL API Token (für Bildgenerierung)

## Flow

1. **Request ohne Payment** → `402 Payment Required`
2. **NFT Mint** (außerhalb Notebook)
3. **Request mit Payment Proof** → `200 OK` + Bildgenerierung

## Python x402 Package

**Verfügbar:** `pip install x402` (v1.0.0, by Coinbase)

**Einschränkung:** Das Package ist für **Facilitator-basierte Workflows** mit **Token-Transfers** (USDC etc.) ausgelegt:
- `x402.clients.requests.x402_requests()`: Automatisches Signing von Token-Transfers
- `x402.clients.base.x402Client`: Payment selector für ERC-20 Tokens
- Unterstützte Schemes: Token-Transfer über Facilitator

**Unser Use Case:** NFT Mint (ERC-721), NICHT Token-Transfer (ERC-20)
→ Package nicht direkt anwendbar

**Alternative:** Direkte HTTP Requests mit `requests` library

In [11]:
import requests
from pprint import pprint

In [7]:
PROMPT = "A small furry elephant pet looks out from a cat house"
DEMO_TX_HASH = "0x1234567890abcdef1234567890abcdef1234567890abcdef1234567890abcdef"
# API Endpoint
BASE_URL = "http://localhost:8081"

print("🚀 x402 Payment Flow Demo")
print(f"📍 Target: {BASE_URL}")
print(f"✍️  Prompt: {PROMPT}\n")
payment_proof = {"txHash": DEMO_TX_HASH}

🚀 x402 Payment Flow Demo
📍 Target: http://localhost:8081
✍️  Prompt: A small furry elephant pet looks out from a cat house



In [12]:
# Schritt 3: Zweite Anfrage MIT Payment Proof
print("\n" + "=" * 60)
print("SCHRITT 3: Request mit Payment Proof")
print("=" * 60)

payload_with_payment = {
    "prompt": PROMPT,
    "payment": payment_proof
}

try:
    print("\n📤 Sende Anfrage mit Payment Proof...")
    response2 = requests.post(BASE_URL, json=payload_with_payment, timeout=30)
    
    print(f"📬 Status Code: {response2.status_code}\n")
    
    if response2.status_code == 200:
        print("✅ ERFOLG! Bildgenerierung gestartet!")
        result = response2.json()
        print("\n🎨 Generierungsergebnis:")
        pprint(result, width=80, indent=2)
        
        if 'image_url' in result:
            print(f"\n🖼️  Bild URL: {result['image_url']}")
        if 'metadata_url' in result:
            print(f"📋 Metadata: {result['metadata_url']}")
        if 'tokenId' in result:
            print(f"🎫 Token ID: {result['tokenId']}")
            
    elif response2.status_code == 402:
        print("❌ Payment Verification fehlgeschlagen")
        error = response2.json()
        print("\n📄 Fehlermeldung:")
        pprint(error, width=80, indent=2)
        
        print("\n💡 Mögliche Ursachen:")
        print("   • Transaction Hash ist nicht real (Demo-Hash)")
        print("   • Transaction ist noch nicht confirmed")
        print("   • Transaction ist kein Mint an den richtigen Contract")
        print("   • Mint-Price war zu niedrig")
        
    else:
        print(f"❌ Unerwarteter Status: {response2.status_code}")
        print("Response:")
        print(response2.text[:500])
        
except requests.exceptions.Timeout:
    print("⏱️  Timeout - Bildgenerierung dauert ~10-30 Sekunden")
    print("   Server arbeitet möglicherweise noch...")
except requests.exceptions.ConnectionError:
    print("❌ Verbindungsfehler! Server nicht erreichbar.")
except Exception as e:
    print(f"❌ Error: {e}")


SCHRITT 3: Request mit Payment Proof

📤 Sende Anfrage mit Payment Proof...
📬 Status Code: 402

❌ Payment Verification fehlgeschlagen

📄 Fehlermeldung:
{ 'error': 'Payment verification failed',
  'message': 'Verification failed: Transaction receipt with hash '
             '"0x1234567890abcdef1234567890abcdef1234567890abcdef1234567890abcdef" '
             'could not be found. The Transaction may not be processed on a '
             'block yet.\n'
             '\n'
             'Version: viem@2.38.3'}

💡 Mögliche Ursachen:
   • Transaction Hash ist nicht real (Demo-Hash)
   • Transaction ist noch nicht confirmed
   • Transaction ist kein Mint an den richtigen Contract
   • Mint-Price war zu niedrig


## Schritt 3: Zweite Anfrage mit Payment Proof

Jetzt senden wir die gleiche Anfrage mit dem Payment Proof.

In [ ]:
# Für echten Test: Ersetze mit deinem Mint Transaction Hash
# Example: Nach NFT Mint auf Optimism erhältst du einen txHash

# DEMO: Verwende einen simulierten txHash (wird von Server abgelehnt)
# Für echten Test: Mint ein NFT und verwende den echten txHash
DEMO_TX_HASH = "0x1234567890abcdef1234567890abcdef1234567890abcdef1234567890abcdef"

payment_proof = {
    "txHash": DEMO_TX_HASH
}

print("\n" + "=" * 60)
print("SCHRITT 2: Payment Proof vorbereiten")
print("=" * 60)
print("\n💳 Payment Proof:")
pprint(payment_proof, width=80, indent=2)

print("\n⚠️  HINWEIS:")
print("   Dieser Demo-Hash ist NICHT real und wird abgelehnt.")
print("   Für echten Test:")
print("   1. Gehe zu https://fretchen.github.io/imagegen")
print("   2. Minte ein NFT")
print("   3. Kopiere den Transaction Hash")
print("   4. Ersetze DEMO_TX_HASH oben")

## Schritt 2: NFT Mint (außerhalb Notebook)

In der Realität würde jetzt:
1. User verbindet Wallet (MetaMask)
2. User mintet NFT via Contract: `0x80f95d330417a4acEfEA415FE9eE28db7A0A1Cdb`
3. Transaction wird confirmed
4. User erhält `txHash` und `tokenId`

**Für Demo:** Wir verwenden einen realen Transaction Hash von Optimism.

**Echte Mint-Transaction auf Optimism:**
- Contract: GenImNFTv4 auf Optimism
- Beispiel TX: https://optimistic.etherscan.io/tx/0x...

In [ ]:
# Schritt 1: Erste Anfrage OHNE Payment
print("=" * 60)
print("SCHRITT 1: Request ohne Payment (sollte 402 zurückgeben)")
print("=" * 60)

payload = {
    "prompt": PROMPT
}

try:
    response = requests.post(BASE_URL, json=payload, timeout=5)
    
    print(f"\n📬 Status Code: {response.status_code}")
    
    if response.status_code == 402:
        print("✅ Erwartete 402 Payment Required Response erhalten!\n")
        
        # Parse Response Body
        body = response.json()
        print("📄 Response Body:")
        pprint(body, width=80, indent=2)
        
        # Extract X-Payment header
        x_payment_header = response.headers.get('X-Payment', response.headers.get('x-payment'))
        if x_payment_header:
            print("\n🔗 X-Payment Header (Payment Instructions):")
            payment_instructions = json.loads(x_payment_header)
            pprint(payment_instructions, width=80, indent=2)
            
            # Store for next step
            contract_address = payment_instructions.get('recipient')
            mint_price = payment_instructions.get('maxAmountRequired')
            print(f"\n💡 To proceed:")
            print(f"   - Mint NFT at: {contract_address}")
            print(f"   - Cost: {mint_price} wei ({int(mint_price) / 1e18:.6f} ETH)")
        else:
            print("\n⚠️  Kein X-Payment Header gefunden!")
    else:
        print(f"❌ Unerwarteter Status: {response.status_code}")
        print(f"Response: {response.text[:200]}")
        
except requests.exceptions.ConnectionError:
    print("❌ Verbindungsfehler!")
    print("   Ist der Server gestartet? → cd scw_js && npm run dev:x402")
except Exception as e:
    print(f"❌ Error: {e}")

## Schritt 1: Erste Anfrage ohne Payment

Die API sollte eine 402 Payment Required Response zurückgeben.

In [ ]:
import requests
import json
from pprint import pprint

